In [ ]:
!pip install pytorch-lightning

In [ ]:
# Importing libraries 
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader,Dataset

In [ ]:
import torch
import pytorch_lightning as pl

class MyModule(ivy.Module):

    def __init__(self):
        super().__init__()
        self.linear1 = ivy.Linear(28*28, 64)
        self.linear2 = ivy.Linear(64, 10)

    def forward(self, x):
        x = ivy.relu(self.linear1(x))
        x = self.linear2(x)
        return x

class MyLightningModule(pl.LightningModule):

    def __init__(self):
        # Here we Instantiate all our models
        super().__init__()
        self.l1 = ivy.Linear(28 * 28 , 64)
        self.l2 = ivy.Linear(64,64)
        self.l3 = ivy.Linear(64,10)
        self.do = ivy.Dropout(0.1) # Dropout can drastically reduce the chance of overfitting
        self.loss = ivy.CrossEntropyLoss()

      # Define the network structure
    def forward(self , x):
        h1 = ivy.functional.relu(self.l1(x))
        h2 = ivy.functional.relu(self.l2(h1))
        do = self.do(h2+h1)
        logits = self.l3(do) # Return raw values for our predictions
        return logits

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = ivy.binary_cross_entropy_with_logits(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    def prepare_data(self):
        datasets.MNIST('data' , train = True , download= True, transform= transforms.ToTensor())

    def setup(self,stage = None):
        dataset = datasets.MNIST('data' , train = True , download= False, transform= transforms.ToTensor())
        self.train , self.val = random_split(dataset , [55000 ,5000])

    def train_dataloader(self):   
        # Train and load data [Train,Val, data in Pytorch]
        train_loader = DataLoader(self.train , batch_size = 32)
        return train_loader

    def val_loader(self):
        val_loader = DataLoader(self.val , batch_size = 32)
        return val_loader

In [ ]:
from pytorch_lightning.callbacks.progress.base import ProgressBarBase
# Training and fitting the model
trainer = pl.Trainer(enable_progress_bar= 20 , max_epochs= 5 , gpus = 1)
trainer.fit(model)